Project script is made for reading several .txt files, containing some data in several columns, converting them into valid dataframe and sorting values according to the relevant column.
Files need to be processed and combined into one .csv file, data needs to be wrangled and excluded of Nan values. Text files in this case are made of rows that are separated by points (......).

In [14]:
import os
import pandas as pd

Initializing empty Dataframe and function which will add empty row at the end of each .txt file, since last row is always shown as bad one and Pandas skips it.

In [6]:
combined_files = pd.DataFrame()
def add_rows_to_text_file(file_path):
    with open(file_path, 'a') as file:
        file.write('\n')

In [60]:
directory = "C:/Users\Anja/Downloads/relative importance inputs/No item code"

Processing/ reading files into Dataframe, skipping header rows (12) and creating rows with tab as delimiter. All of files are then concating into one Dataframe. Each row is then split and added into combined data.

In [89]:

def process_and_combine_files(directory):
    combined_data = pd.DataFrame()  # Initialize an empty DataFrame to store the combined data

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            add_rows_to_text_file(file_path)
            data = pd.read_csv(file_path, delimiter="\t", on_bad_lines='skip', skiprows=12)
            data.columns = ['Podaci']
            splitted = data['Podaci'].str.split(r'\.{2,}', n=1, expand=True)
            data[['Item and Group', 'Value']] = splitted
            data[['CPI-U', 'CPI-W']] = data['Value'].str.split(expand=True)
            data.drop(columns=['Podaci', 'Value'], inplace=True)
            data['Item and Group'] = data['Item and Group'].str.strip()
            data['CPI-U'] = data['CPI-U'].str.strip()
            data['CPI-W'] = data['CPI-W'].str.strip()

            combined_data = pd.concat([combined_data,data], ignore_index=True)

    return combined_data

In [105]:
df = process_and_combine_files(directory)
df.reset_index(drop = True, inplace=True)
df.shape

(7753, 3)

Combining files created several rows that had Nan values (usually data that was represented  in 2 rows). Correcting rows is merging 2 rows into one, combining first column (string) into one string and merging float values into one value. Then, I erased rows which had all Nan values in second and third column.

In [106]:
for i in range(len(df) - 1):
    if pd.isna(df.at[i, 'CPI-U']) and pd.isna(df.at[i, 'CPI-W']):
        df.at[i+1, 'Item and Group'] = df.at[i, 'Item and Group'] + ' ' + df.at[i+1, 'Item and Group']
        df.at[i, 'Item and Group'] = ''

df.isna().sum()
df = df[df['Item and Group'] != '']
df.shape

(7361, 3)

In [107]:
out_dir_proba = "E:/"
out_csv = os.path.join(out_dir_proba, 'combined_part2.csv')
df.to_csv(out_csv, index=False)